In [1]:
from datetime import datetime
from os.path import getmtime

PATH_EVENTS = '../data/events.csv'

print(f'Last modified on {datetime.fromtimestamp(getmtime(PATH_EVENTS)).strftime("%Y-%m-%d %H:%M:%S")}')

with open(PATH_EVENTS, 'r', encoding='utf-8') as fp:
	data = [event.split(',') for event in fp.read().strip().split('\n')]

print(f'Loaded {len(data)} events')

Last modified on 2024-03-08 01:58:52
Loaded 3850585 events


In [2]:
PATH_IDS = '../data/ids.txt'

print(f'Last modified on {datetime.fromtimestamp(getmtime(PATH_IDS)).strftime("%Y-%m-%d %H:%M:%S")}')

with open(PATH_IDS, 'r', encoding='utf-8') as fp:
	ids = fp.read().strip().split('\n')

print(f'Loaded {len(ids)} ids')

Last modified on 2023-10-13 00:21:46
Loaded 29636 ids


In [5]:
KINDS = {
	'B': {'name': 'begin', 'fields': {}},
	'W': {'name': 'window', 'fields': {'app': 'id', 'activity': 'id', 'desktop': 'string'}},
	'I': {'name': 'idle', 'fields': {'enabled': 'bool'}},
	'K': {'name': 'keyboard', 'fields': {'key': 'char', 'elapsed': 'float'}},
	'C': {'name': 'click', 'fields': {'x_released': 'int', 'y_released': 'int', 'x_pressed': 'int', 'y_pressed': 'int', 'button': 'int', 'elapsed': 'float'}},
	'S': {'name': 'scroll', 'fields': {'x_start': 'int', 'y_start': 'int', 'direction': 'string', 'lines': 'float', 'elapsed': 'float'}}
}

def parse_event(event):
	res = {'kind': KINDS[event[0]]['name'], 'when': float(event[1])}
	assert len(event) == 2 + len(KINDS[event[0]]['fields']), f'Invalid event: {event}'
	for value, kind in zip(event[2:], KINDS[event[0]]['fields']):
		if KINDS[event[0]]['fields'][kind] == 'string':
			res[kind] = value
		elif KINDS[event[0]]['fields'][kind] == 'int':
			res[kind] = int(value)
		elif KINDS[event[0]]['fields'][kind] == 'float':
			res[kind] = float(value)
		elif KINDS[event[0]]['fields'][kind] == 'bool':
			res[kind] = bool(int(value))
		elif KINDS[event[0]]['fields'][kind] == 'char':
			res[kind] = chr(int(value))
		elif KINDS[event[0]]['fields'][kind] == 'id':
			res[kind] = ids[int(value)]
	return res

events = [parse_event(event) for event in data]

seen = {}
for event in events:
	if event['kind'] in seen:
		seen[event['kind']] += 1
	else:
		seen[event['kind']] = 1
		print(event)
	
print(seen)

{'kind': 'begin', 'when': 0.0}
{'kind': 'window', 'when': 0.245, 'app': '\\Device\\HarddiskVolume4\\Windows\\SystemApps\\Microsoft.LockApp_cw5n1h2txyewy\\LockApp.exe', 'activity': 'Windows Default Lock Screen', 'desktop': '0'}
{'kind': 'click', 'when': 38.25, 'x_released': -764, 'y_released': 1014, 'x_pressed': -764, 'y_pressed': 1014, 'button': 1, 'seconds': 0.02951169}
{'kind': 'keyboard', 'when': 41.797, 'key': 'b', 'seconds': 0.08000088}
{'kind': 'scroll', 'when': 70.18, 'x_start': 3227, 'y_start': 295, 'direction': 'U', 'amount': 7.0, 'seconds': 2.24640489}
{'kind': 'idle', 'when': 252.596, 'enabled': True}
{'begin': 100, 'window': 190866, 'click': 773242, 'keyboard': 2760378, 'scroll': 106169, 'idle': 19830}


In [27]:
from collections import Counter

def event_times(kind, field):
	res = {}
	for event in events:
		if event['kind'] == 'begin':
			current_value, current_start = None, None
		elif event['kind'] == kind:
			if event[field] not in res: res[event[field]] = 0
			if current_value != event[field]:
				if current_value is not None:
					res[current_value] += event['when'] - current_start
				current_value, current_start = event[field], event['when']
	return res

top_apps = event_times('window', 'app')
total_time = sum(top_apps.values())

print('Top 50 apps by time spent:')
for name, time in Counter(top_apps).most_common(50):
	share = time / total_time
	print('% 5dh\t% 7.2f %%\t%s' % (time / 3600, 100 * share, name.rsplit('.', 1)[0].rsplit('/', 1)[-1].rsplit('\\', 1)[-1]))

Top 50 apps by time spent:
 1430h	  68.07 %	
  436h	  20.79 %	chrome
   84h	   4.01 %	LockApp
   27h	   1.33 %	Around
   21h	   1.04 %	hyper
   18h	   0.89 %	Telegram
   15h	   0.76 %	explorer
   15h	   0.73 %	OUTLOOK
   12h	   0.60 %	factorio
    9h	   0.46 %	sublime_text
    8h	   0.42 %	GameMaker
    8h	   0.41 %	Code - Insiders
    8h	   0.40 %	WINWORD
    8h	   0.38 %	photoshop
    7h	   0.35 %	EXCEL
    6h	   0.31 %	POWERPNT
    6h	   0.30 %	PUTTY
    5h	   0.28 %	Spotify
    5h	   0.25 %	Ableton Live 11 Suite
    5h	   0.25 %	openscad
    3h	   0.15 %	blender
    2h	   0.14 %	Cura
    2h	   0.13 %	mspaint
    1h	   0.09 %	Runner
    1h	   0.09 %	firefox
    1h	   0.08 %	SearchApp
    1h	   0.08 %	Aseprite
    1h	   0.07 %	ShellExperienceHost
    1h	   0.07 %	SumatraPDF
    1h	   0.06 %	Runner
    1h	   0.05 %	illustrator
    1h	   0.05 %	WinSCP
    1h	   0.05 %	ScreenClippingHost
    1h	   0.05 %	AcroRd32
    1h	   0.05 %	Zoom
    0h	   0.05 %	qbittorrent
    0h	   0.05 %	HotDig

In [28]:
top_activities = event_times('window', 'activity')
total_time = sum(top_activities.values())
print('Top 50 activities by time spent:')
for name, time in Counter(top_activities).most_common(50):
	share = time / total_time
	print('% 5dh\t% 7.2f %%\t%s' % (time / 3600, 100 * share, name))

Top 50 activities by time spent:
 1264h	  60.13 %	
   84h	   4.01 %	Windows Default Lock Screen
   84h	   4.00 %	Change the date and time
   26h	   1.24 %	Around
   21h	   1.04 %	Moda primaveral serena: Urban Classics | ABOUT YOU - Google Chrome
   12h	   0.61 %	rafi.wadan - Stargazr Chat - Google Chrome
   12h	   0.61 %	HyperRogue 12.1o
   12h	   0.60 %	Factorio 1.1.80
   10h	   0.50 %	● main.ipynb - Stargazr (Workspace) - Visual Studio Code [Administrator]
    7h	   0.37 %	HyperRogue 12.1p
    5h	   0.28 %	Packing list & planning - Carlos - Google Docs - Google Chrome
    5h	   0.27 %	New Tab - Google Chrome
    5h	   0.24 %	Inbox - jc.roldan@stargazr.ai - Outlook
    4h	   0.23 %	● 2 Computation.ipynb - Visual Studio Live Share (Workspace) - Visual Studio Code [Administrator]
    4h	   0.22 %	ChatGPT - Google Chrome
    4h	   0.21 %	How does the computational engine works? - Google Chrome
    4h	   0.20 %	Book1 - Excel
    4h	   0.20 %	Dev - Stargazr Chat - Google Chrome
    4h	   0

In [29]:
CHROME_SUFFIX = ' - Google Chrome'
top_chrome = {k[:-len(CHROME_SUFFIX)]: v for k, v in top_activities.items() if k.endswith(CHROME_SUFFIX)}
total_time = sum(top_chrome.values())
print('Top 50 Chrome tabs by time spent:')
for name, time in Counter(top_chrome).most_common(50):
	share = time / total_time
	print('% 5dh\t% 7.2f %%\t%s' % (time / 3600, 100 * share, name))

Top 50 Chrome tabs by time spent:
   21h	   5.01 %	Moda primaveral serena: Urban Classics | ABOUT YOU
   12h	   2.95 %	rafi.wadan - Stargazr Chat
    5h	   1.32 %	Packing list & planning - Carlos - Google Docs
    5h	   1.30 %	New Tab
    4h	   1.06 %	ChatGPT
    4h	   1.01 %	How does the computational engine works?
    4h	   0.98 %	Dev - Stargazr Chat
    3h	   0.91 %	manuel.barea - Stargazr Chat
    3h	   0.88 %	Home • Stargazr
    3h	   0.79 %	pablo.huet - Stargazr Chat
    3h	   0.73 %	jcarlosroldan.com
    2h	   0.68 %	● Dev - Asana
    2h	   0.68 %	The Best Free Plugins for Ableton Live – Abletunes Blog | Music Production in Ableton Live
    2h	   0.65 %	Feed | LinkedIn
    2h	   0.61 %	WhatsApp
    2h	   0.60 %	General - Stargazr Chat
    2h	   0.60 %	🌴👾🌴 Stargazr tech vacations - Google Sheets
    2h	   0.59 %	Unsaved YTD Value Driver • Stargazr
    2h	   0.54 %	2vdiagram.gif (431×340)
    2h	   0.54 %	Stargazr Clocking
    2h	   0.52 %	How does the computational engine work?
 

In [56]:
text = ''
ALLOW = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789.-,;:!?()[]{}<>/\\\'"  '
for event in events:
	if event['kind'] == 'keyboard':
		if event['key'] in ALLOW:
			text += event['key']
		# backspace:
		elif event['key'] == '\x08':
			text = text[:-1]
		else:
			text += ' '

In [77]:
remaining_text = text.lower()
SEARCH = 'twitter'
while SEARCH in remaining_text:
	i = remaining_text.index(SEARCH)
	print(remaining_text[i - 60:i + 60])
	remaining_text = remaining_text[i + len(SEARCH):]

t  pp-level to make it less exaggerated i n the footer, the twitter symbol is lower than the rest  i n s olution, the ta
y nested, like in typical big platforms such as facebook or twitter;  you inspect a single user name there and you'll se
(((   v as a ayudarme a crear textos de resumen para el las twitter cards de mi blog .  e voy a ir pasando post del blog
                                     texto para poner en la twitter card de este post de mi blog personal, tancomo en i.
